In [22]:
#!pip install python-docx

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 239 kB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 5.0 MB 69.8 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [9]:
import os
import sys
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from docx import Document

PROJECT_SRC_PATH = os.path.join( '/workspace/workspace/ufo-prediction', 'src-RCA')
sys.path.append(PROJECT_SRC_PATH)

import visualizations
import dataset

In [12]:
#path_data_RCA = os.path.join(dataset.DATA_DIR, 'rca-ufo-CNN.csv')
path_data_RCA = os.path.join(dataset.DATA_DIR, 'ufo-CNN_ALL.csv')
df = pd.read_csv(path_data_RCA, encoding='latin1')
#df['Zip_cd'] = df['Zip_cd'].astype(str)
#df['NumberFloors_nb'] = pd.to_numeric(df['NumberFloors_nb'], errors='coerce')
df

,id,Class_0,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Predicted Class,source_file,...,street_betweeness_global_max_within_buffer_100,TouchesIndexes,street_based_block_av_phi_inter_buffer_500,total_length_street_city,total_buildings_footprint_city,Phi,street_based_block_std_area_inter_buffer_100,type,neighborhood,_merge
0,v0.1-NLD.1.11_1-19085,0.076568,0.088220,0.082500,0.152224,0.190109,0.167269,0.243111,6,NL_lod12_2d_pand_2,...,0.056066,[19085],0.231087,888712.235310,2.887787e+06,0.441517,4.159843e+06,NaN,547d1be9,both
1,v0.1-NLD.1.2_1-21267,0.071866,0.074957,0.081186,0.095314,0.282351,0.231808,0.162518,4,NL_lod12_2d_pand_2,...,0.019270,[21267],0.378814,708496.163500,3.936964e+06,0.504573,4.913307e+05,NaN,4108f866,both
2,v0.1-NLD.1.2_1-37017,0.044820,0.043458,0.110440,0.112738,0.210129,0.299723,0.178693,5,NL_lod12_2d_pand_2,...,0.058482,[37017],0.340734,708496.163500,3.936964e+06,0.577130,2.582773e+06,NaN,08bd71fe,both
3,v0.1-NLD.1.2_1-37316,0.073977,0.081691,0.128484,0.154770,0.241503,0.175542,0.144033,4,NL_lod12_2d_pand_2,...,0.024308,[37316],0.338987,708496.163500,3.936964e+06,0.355735,9.178285e+05,NaN,b395e25a,both
4,v0.1-NLD.1.2_1-39653,0.150131,0.148781,0.120095,0.101681,0.222674,0.145629,0.111009,4,NL_lod12_2d_pand_2,...,0.009757,[39653],0.423478,708496.163500,3.936964e+06,0.869037,5.446780e+05,NaN,4108f866,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2143,v0.1-FRA.9.2.2.8_1-12267,0.140232,0.103981,0.122269,0.083772,0.165550,0.183416,0.200781,6,BATIMENT-normandie,...,0.088585,"[1302, 1917, 1916, 1348, 12268, 19402, 12266, ...",0.382826,411652.822103,2.715634e+06,0.599850,7.216077e+03,non-residential,3727ea1b,both
2144,v0.1-FRA.9.2.2.8_1-16210,0.044276,0.022045,0.015144,0.040982,0.409258,0.403121,0.065174,4,BATIMENT-normandie,...,0.022207,[16210],0.346928,411652.822103,2.715634e+06,0.386748,2.818122e+04,residential,ab49433b,both
2145,v0.1-FRA.9.3.2.5_1-6845,0.240558,0.199165,0.078928,0.081420,0.132393,0.157868,0.109667,0,BATIMENT-normandie,...,0.105030,"[6844, 6846, 14720, 6845, 14705, 6847]",0.208099,398233.815208,2.162725e+06,0.615855,7.080620e+05,residential,d92b71eb,both
2146,v0.1-FRA.9.3.4.2_1-10576,0.262548,0.145477,0.094362,0.079989,0.127766,0.215046,0.074812,0,BATIMENT-normandie,...,0.020958,"[2610, 9296, 9969, 2613, 2614, 2612, 9297, 105...",0.379129,559702.734166,1.836864e+06,0.358921,1.751522e+04,non-residential,8d4f2f97,both


In [13]:
# Separate numeric and non-numeric (categorical) columns
numeric_cols = df.select_dtypes(include=[np.number])  # Selects all numeric columns
categorical_cols = df.select_dtypes(exclude=[np.number])  # Selects all non-numeric columns

# Summary for numeric columns: default stats including mean, std, etc.
numeric_summary = numeric_cols.describe().round(3).transpose()
numeric_summary.reset_index(inplace=True)
numeric_summary.rename(columns={'index': 'Statistic'}, inplace=True)
numeric_summary.insert(0, 'Data Source', 'Numeric Data')  # Change as necessary
numeric_summary.insert(2, 'Variable Description', 'Numerical Summary')

# Summary for categorical columns: count, unique, top, freq
categorical_summary = categorical_cols.describe(include='all').transpose()
categorical_summary.reset_index(inplace=True)
categorical_summary.rename(columns={'index': 'Statistic'}, inplace=True)
categorical_summary.insert(0, 'Data Source', 'Categorical Data')  # Change as necessary
categorical_summary.insert(2, 'Variable Description', 'Categorical Summary')


In [14]:
from docx import Document

def df_to_word(numeric_df, categorical_df, filename):
    doc = Document()
    
    # Adding Numeric Summary Table
    doc.add_heading('Numeric Summary Statistics', level=1)
    num_table = doc.add_table(rows=(numeric_df.shape[0] + 1), cols=numeric_df.shape[1])
    for i, col_name in enumerate(numeric_df.columns):
        num_table.cell(0, i).text = str(col_name)
    for row_index, row in numeric_df.iterrows():
        for col_index in range(numeric_df.shape[1]):
            num_table.cell(row_index + 1, col_index).text = str(row[col_index])
    num_table.style = 'Table Grid'
    
    # Adding Categorical Summary Table
    doc.add_heading('Categorical Summary Statistics', level=1)
    cat_table = doc.add_table(rows=(categorical_df.shape[0] + 1), cols=categorical_df.shape[1])
    for i, col_name in enumerate(categorical_df.columns):
        cat_table.cell(0, i).text = str(col_name)
    for row_index, row in categorical_df.iterrows():
        for col_index in range(categorical_df.shape[1]):
            cat_table.cell(row_index + 1, col_index).text = str(row[col_index])
    cat_table.style = 'Table Grid'

    # Save the document
    doc.save(filename)

# Example usage
df_to_word(numeric_summary, categorical_summary, '/workspace/workspace/ufo-prediction/main notebooks/summary_statistics.docx')



# Descriptive statistics

In [ ]:
import matplotlib.pyplot as plt

# Filter out the 'No Match' category from the 'match_type' column
filtered_matches = matches[matches['match_type'] != 'no match']

# Create the bar plot and get the axis object
ax = filtered_matches['match_type'].value_counts().plot(kind='bar')

plt.xlabel('Match Type')
plt.ylabel('Count')
plt.title('Distribution of Match Types')

# Adding the text on top of each bar
for bar in ax.patches:
    # The text to display (bar height)
    yval = bar.get_height()
    # Placing the text above the bar
    ax.text(bar.get_x() + bar.get_width() / 2, yval, int(yval), va='bottom', ha='center')

plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'matches' is your DataFrame and it has a numerical column named 'distances'
matches['distances'].plot(kind='hist', bins=20)  # You can adjust the number of bins

plt.xlabel('Distance')
plt.ylabel('Frequency')
plt.title('Histogram of Distances')
plt.show()

In [ ]:
    # Idea: periods with more buildings will not be brighter than periods with little buildings
import matplotlib
from matplotlib.gridspec import GridSpec

bins = list(range(min_age, max_age + 1, 5))
ticks = [1920, 1940, 1960, 1980, 2000]
X, Y = np.meshgrid(bins, bins)
age_ufo = matches['age_right']
age_rca = matches['YearBlt']
H = np.histogram2d(age_ufo, age_rca, bins=bins)[0]
# H_norm: each row describes relative share of all prediction age bands for buildings of a certain test band
H_norm = (H.T / H.sum(axis=1)).T
#H_norm[np.isinf(H_norm)] = 0
#H_norm[np.isnan(H_norm)] = 0

fig = plt.figure(figsize=(10, 10))
gs = GridSpec(4, 4)

ax_scatter = fig.add_subplot(gs[1:4, 0:3])
ax_hist_x = fig.add_subplot(gs[0,0:3])
ax_hist_y = fig.add_subplot(gs[1:4, 3])

cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ['white', 'cadetblue'])
colors = ['thistle', 'lightsteelblue', 'plum']

ax_scatter.plot([0, 1], [0, 1], transform=ax_scatter.transAxes, color='cadetblue')
ax_scatter.pcolormesh(X, Y, H_norm, cmap=cmap, rasterized=True)
ax_scatter.set_yticks(ticks, labels=ticks)
ax_scatter.set_xticks(ticks, labels=ticks)


ax_hist_x.hist(matches['YearBlt'], bins=bins, color=colors[0], alpha=0.4)
ax_hist_y.hist(matches['age_right'], bins=bins, orientation='horizontal', color=colors[0], alpha=0.4)


ax_hist_x.set_axis_off()
ax_hist_y.set_axis_off()
fig.subplots_adjust(hspace=0.05, wspace=0.02)

ax_scatter.spines['top'].set_linewidth(0.5)
ax_scatter.spines['right'].set_linewidth(0.5)
ax_scatter.spines['left'].set_linewidth(0.5)
ax_scatter.spines['bottom'].set_linewidth(0.5)

ax_scatter.set_xlabel('RCA construction year')
ax_scatter.set_ylabel('UFO construction year')
plt.show()

In [ ]:
class SubplotManager:
    def __init__(self, **kwargs):
        ax = kwargs.get('ax')
        self.mainplot = not bool(ax)
        self.ax = ax or plt.subplots(**{'figsize': (12, 12), **kwargs})[1]

        SMALL_SIZE = 10
        MEDIUM_SIZE = 12
        BIGGER_SIZE = 14
        # SMALL_SIZE = 18
        # MEDIUM_SIZE = 20
        # BIGGER_SIZE = 22

        plt.rc('font', size=BIGGER_SIZE)
        plt.rc('axes', titlesize=BIGGER_SIZE)
        plt.rc('axes', labelsize=MEDIUM_SIZE)
        plt.rc('xtick', labelsize=MEDIUM_SIZE)
        plt.rc('ytick', labelsize=MEDIUM_SIZE)
        plt.rc('legend', fontsize=SMALL_SIZE)
        plt.rc('figure', titlesize=BIGGER_SIZE)


    def __enter__(self):
        return self.ax


    def __exit__(self, exc_type, exc_value, exc_traceback):
        if self.mainplot:
            plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
def plot_histogram(y_test, y_predict, bins=None, bin_labels=[], **kwargs):
    with SubplotManager(**kwargs) as ax:
        if bin_labels:
            ax.set_xticklabels([None] + bin_labels)

        sns.histplot(
            y_predict,
            ax=ax,
            kde=False,
            line_kws=dict(edgecolor="k", linewidth=1),
            palette='Set2',
            bins=bins,
            label='UFO construction year'
        )
        sns.histplot(
            y_test,
            ax=ax,
            kde=False,
            line_kws=dict(edgecolor="k", linewidth=1),
            palette='husl',
            bins=bins,
            label='RCA construction year'
        )
        ax.legend()
        ax.set_title('age distributions')

In [ ]:
#YearBlt is y_test
# Filter out the rows where 'match_type' is 'no match'
matches_new = matches[matches['match_type'] != 'no match']
matches_new2 = matches_new[matches_new['YearBlt'].notna()]
plot_histogram(matches_new2['YearBlt'], matches_new2['age_right'], bins=list(range(min_age, max_age + 1))[0::1])

ESP <br>
Total buildings in ESP in RCA: 3934 <br>
Total buildings in ESP in RCA with age:  <br>
Total matches: 2906 <br>
Matches after filtering those that have distance > footprint length: 1747 <br>
Matches after filtering for age differences > 10 years: 1482 <br>
Of which YearBlt is defined: 549 <br>

FRA <br>
Total buildings in FRA in RCA: 10476 <br>
Total buildings in FRA in RCA with age: 6393 <br>
Total matches: 3217, there is a lot of matches where the RCA year is near 2020, but the ufo one is much older. Youngest ufo building is 2017 which is also strange <br>
Matches after filtering those that have distance > footprint length: 1100 <br>
Matches after filtering for age differences > 10 years:  870 <br>
Of which YearBlt is defined: 408 <br>



NLD <br>
Total buildings in NLD in RCA: 10136 <br>
Total buildings in NLD in RCA with age: 8050 <br>
Total matches: 7318, there is a lot of matches where the RCA year is near 2020, but the ufo one is much older. Youngest ufo building is 2020 which is also strange <br>
Matches after filtering those that have distance > footprint length: 4560 <br>
Matches after filtering for age differences > 10 years: 3789 <br>
Of which YearBlt is defined: 3211 <br>

In [ ]:
from sklearn import metrics

print('MAE: {:.2f} y'.format(metrics.mean_absolute_error(matches_new2['age_right'], matches_new2['YearBlt'])))
print('RMSE: {:.2f} y'.format(np.sqrt(metrics.mean_squared_error(matches_new2['age_right'], matches_new2['YearBlt']))))
print('R2: {:.4f}'.format(metrics.r2_score(matches_new2['YearBlt'], matches_new2['age_right'])))